In [102]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
ranges = [(9, 13),     # XLPE/EVA
    (18.957, 19, 357),
    (13.879, 14.103),  # Graphite
    (11.017, 11.068),  # Mica
    (12.226, 12.396),
    (13.426, 13.603),
    (16.484, 16.660),
    (14.393, 14.793),  # NaCl
    (16.663, 17.063),
    (23.733, 24.133),
    (27.933, 28.333),
    (29.213, 29.613),
    (33.950, 34.250),
    (37.043, 37.443),
    (38.048, 38.448),
    (41.940, 42.200),
    (44.630, 44.859),
    (14.993, 15.173),  # KCl
    (21.303, 21.484),
    (25.169, 25.324),
    (26.124, 26.472),
    (15.522, 15.803),  # CaCO3
    (19.054, 19.154),
    (20.720, 20.948),
    (22.690, 22.815),
    (24.650, 24.812),
    (25.328, 25.596),
    (29.687, 29.917),
    (31.276, 31.910),
    (33.147, 33.501),
    (33.753, 33.946),
    (19.154, 19.368),  # ZnO
    (18.193, 18.464),
    (24.873, 25.117),
    (32.292, 32.617),
    (34.783, 35.007),
    (35.227, 35.568),
    (38.717, 39.275),
    (44.330, 44.574),
    (14.45, 14.70),    #Unknowns
    (28.3, 28.5),
    (26.55, 26.85),
    (13, 13.2),
    (17.75, 17.95),
    (18.3, 18.7),
    (24.26, 24.80),
    (28.9, 29.10)
              ]

In [121]:
from beavermap import BeaverMap
import os 
bm = BeaverMap(
    h5_file = './data/al2o3_m330p0/al2o3_m330p0.h5',
    poni_file = './data/m330p0.poni',
    mask_file = './data/mask.edf',
    chunk_size = 100,
    location = '1.1/measurement/eiger',
    nworkers = 1,
    max_memory = 4000
)
bm.default_integrate_args


In [ ]:
results = bm._pathos_tqdm_integrate(bm.integrate_args,regions=ranges)

  0%|          | 0/7200 [00:00<?, ?it/s]

TypeError: _pathos_tqdm_integrate_worker() got an unexpected keyword argument 'maxtasksperchild'

In [106]:
os.environ['OMP_NUM_THREADS'] = '1'
bm.nworkers = 1
#bm.n_images=600
integration = bm.integrate(
    regions = ranges,integrate_args=bm.integrate_args
)

NameError: name 'Full' is not defined

In [107]:
from pathos.helpers import mp as pmp 

ctx = pmp.get_context('spawn')
in_queue = ctx.Queue()
in_queue.full()

False

In [ ]:
import tqdm_pathos

pool= tqdm_pathos.map()

Signature:
tqdm_pathos.map(
    func,
    iterable,
    *args,
    n_cpus=10,
    pool=None,
    tqdm_kwargs={},
    **kwargs,
)
Docstring: <no docstring>
File:      ~/miniconda3/envs/py312/lib/python3.12/site-packages/tqdm_pathos/tqdm_pathos.py
Type:      function